In [1]:
from astroquery.jplhorizons import Horizons
import numpy as np
import time
from datetime import datetime
from datetime import timedelta
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
from joblib import Parallel, delayed
import multiprocessing
import pandas as pd
from astropy.time import Time
import itertools
from astropy import units as u
from astropy.coordinates import SkyCoord
import subprocess
import os

gzip was not found on your system! You should solve this issue for astroquery.eso to be at its best!
On POSIX system: make sure gzip is installed and in your path!On Windows: same for 7-zip (http://www.7-zip.org)!


In [2]:
def generatePlots(obj):
    
    i = obj.replace("\n","")
    try:
        obj = Horizons(id=str(i), location='954', epochs={'start':'2019-11-16 22:00', 'stop':'2019-11-17 00:00', 'step':'1m'})
        ephem = obj.ephemerides(quantities='1,8,9,19,20,23,24,25', airmass_lessthan=2, skip_daylight=True).to_pandas()
    except:
        return
        
    i=i.replace("/","").replace("-","").replace(" ","")
    ephem['datetime_str'] = pd.to_datetime(ephem['datetime_str'],infer_datetime_format=True)

    degsPerMin = (ephem['RA'].iloc[10] - ephem['RA'].iloc[0])/10

    rate = []
    for j in range(10, len(ephem.index), 10):
        rate.append((ephem['RA'].iloc[j] - ephem['RA'].iloc[j-10])/10)

    df = pd.DataFrame(rate)
    averageRate = df[0].mean()
    newEphem = ephem[ephem.V < 17]
    
    if(len(newEphem.index) > 90):
        calcRAStart = [newEphem['RA'].iloc[0]]
        calcRAMean = [newEphem['RA'].iloc[0]]
        for k in range(0, len(ephem.index)-1):
            #calcRAStart.append(newEphem['RA'].iloc[0] + (k*degsPerMin))
            #calcRAMean.append(newEphem['RA'].iloc[0] + (k*averageRate))
            calcRAStart.append(calcRAStart[k] + degsPerMin)
            calcRAMean.append(calcRAMean[k] + averageRate)

        newEphem['calcRAStart'] = calcRAStart
        newEphem['calcRAMean'] = calcRAMean
        newEphem['startRes'] = newEphem['RA'] - newEphem['calcRAStart']
        newEphem['meanRes'] = newEphem['RA'] - newEphem['calcRAMean']
        newEphem['zero'] = newEphem['RA'] - newEphem['RA']
        rmseStart = np.sqrt(np.square(newEphem['startRes']).sum())
        rmseMean = np.sqrt(np.square(newEphem['meanRes']).sum())
        fig = plt.figure(figsize=(10,8))
        frame1=fig.add_axes((.1,.3,.8,.6))
        ax=plt.gca()

        newEphem.plot(x='datetime_str', y='RA', ax=ax, label='RA from JPL HORIZONS')
        newEphem.plot(x='datetime_str', y='calcRAStart', ax=ax, label='Calculated RA using initial tracking rate')
        newEphem.plot(x='datetime_str', y='calcRAMean', ax=ax, label='Calculated RA using mean tracking rate for night')
        ax.set_ylabel("Right Ascension [degrees]")
        ax.xaxis.set_tick_params(labelbottom=False, which='both', direction='in', labelsize=14)

        frame2=fig.add_axes((.1,.1,.8,.2))
        ax = plt.gca()
        ax.set_xlabel("Universal Time [UT]")
        newEphem.plot(x='datetime_str', y='zero', ax=ax, label='Root Mean Square Residual = 0.00000')
        newEphem.plot(x='datetime_str', y='startRes', ax=ax, label='Root Mean Square Residual = %.5f' % rmseStart)
        newEphem.plot(x='datetime_str', y='meanRes', ax=ax, label='Root Mean Square Residual = %.5f' % rmseMean)
        ax.set_xlabel("Universal Time [UT]")
        ax.set_ylabel("Residuals [degrees]")
        plt.savefig("trackingRates_%s.pdf" % i, bbox_inches='tight')
        plt.close()
        
        rmseFile = open("temp_%s.csv" % i, "w")
        rmseFile.write("updateTime,rmse\n")
        
        for updateTime in (1,10,20,30,40,50,60):
        
            rate = []
            for j in range(0, len(newEphem.index)-updateTime, updateTime):
                rate.append((newEphem['RA'].iloc[j+updateTime] - newEphem['RA'].iloc[j])/updateTime)
            endIndex = len(newEphem.index)-1
            rate.append((newEphem['RA'].iloc[endIndex] - newEphem['RA'].iloc[endIndex-updateTime])/updateTime)

            calcRA = [newEphem['RA'].iloc[0]]
            for k in range(0, len(newEphem.index)-1):
                calcRA.append(calcRA[k] + rate[k//updateTime])


            newEphem['calcRA'] = calcRA
            newEphem['Res'] = newEphem['RA'] - newEphem['calcRA']
            newEphem['zero'] = newEphem['RA'] - newEphem['RA']
            rmse = np.sqrt(np.square(newEphem['Res']).sum())
            fig = plt.figure(figsize=(10,8))
            frame1=fig.add_axes((.1,.3,.8,.6))
            ax=plt.gca()

            newEphem.plot(x='datetime_str', y='RA', ax=ax, label='RA from JPL HORIZONS')
            newEphem.plot(x='datetime_str', y='calcRA', ax=ax, label='Calculated RA using tracking rate updated every %s mins' % updateTime)
            ax.set_ylabel("Right Ascension [degrees]")
            ax.xaxis.set_tick_params(labelbottom=False, which='both', direction='in', labelsize=14)

            frame2=fig.add_axes((.1,.1,.8,.2))
            ax = plt.gca()
            ax.set_xlabel("Universal Time [UT]")
            newEphem.plot(x='datetime_str', y='zero', ax=ax, label='Root Mean Square Residual = 0.00000')
            newEphem.plot(x='datetime_str', y='Res', ax=ax, label='Root Mean Square Residual = %.5f' % rmse)
            ax.set_xlabel("Universal Time [UT]")
            ax.set_ylabel("Residuals [degrees]")
            plt.savefig("trackingRatesUpdating_%s_%s.pdf" % (i, updateTime), bbox_inches='tight')
            plt.close()

            rmseFile.write("%s,%s\n" % (updateTime,rmse))
            
        rmseFile.close()
        
        df = pd.read_csv(rmseFile.name)
        fig, ax = plt.subplots(figsize=(10,8))
        ax = plt.gca()
        df.plot(x='updateTime', y='rmse', ax=ax, color='k', marker='o', label='RMS Residual for varying update time')
        ax.axhline(y=rmseStart, color='k', linestyle='-.', label='RMS Residual [starting tracking rate]')
        ax.axhline(y=rmseMean, color='k', linestyle='--', label='RMS Residual [mean tracking rate]')
        ax.set_xlabel("Update Time [minutes]")
        ax.set_ylabel("RMS Residual")
        ax.set_yscale('log')
        plt.legend()
        plt.savefig("rmsePlot_%s.pdf" % i, bbox_inches='tight')
        plt.close()
        os.remove(rmseFile.name)


In [4]:
def generatePlotsRates(obj, start, end, programLength):
    
    if not os.path.exists('%sHrProgram/' % programLength):
        os.mkdir('%sHrProgram/' % programLength)
    
    i = obj.replace("\n","")
    try:
        obj = Horizons(id=str(i), location='954', epochs={'start':start, 'stop':end, 'step':'1m'})
        ephem = obj.ephemerides(quantities='1,8,9,19,20,23,24,25', airmass_lessthan=2, skip_daylight=True).to_pandas()
    except:
        return
        
    i=i.replace("/","").replace("-","").replace(" ","")
    ephem['datetime_str'] = pd.to_datetime(ephem['datetime_str'],infer_datetime_format=True)
    newEphem = ephem[ephem.V < 17]
    rate = []
    for j in range(0, len(newEphem.index)-1, 1):
        rate.append((newEphem['RA'].iloc[j+1] - newEphem['RA'].iloc[j])/1)
    endIndex = len(newEphem.index)-1
    rate.append((newEphem['RA'].iloc[endIndex] - newEphem['RA'].iloc[endIndex-1])/1)
        
    newEphem['rate'] = rate
    meanRate = newEphem['rate'].mean()
    startRate = newEphem['rate'].iloc[0]
    newEphem['meanRate'] = meanRate
    newEphem['startRate'] = startRate
    
    newEphem['rate'] = newEphem['rate']*3600
    newEphem['meanRate'] = newEphem['meanRate']*3600
    newEphem['startRate'] = newEphem['startRate']*3600
    
    if(len(newEphem.index) > 59):
    
        newEphem['startRateRes'] = newEphem['rate'] - newEphem['startRate']
        newEphem['meanRateRes'] = newEphem['rate'] - newEphem['meanRate']
        newEphem['zero'] = newEphem['rate'] - newEphem['rate']
        rmseStart = np.sqrt((np.square(newEphem['startRateRes']).sum())/len(newEphem.index))
        rmseMean = np.sqrt((np.square(newEphem['meanRateRes']).sum())/len(newEphem.index))
        fig = plt.figure(figsize=(10,8))
        frame1=fig.add_axes((.1,.3,.8,.6))
        ax=plt.gca()

        newEphem.plot(x='datetime_str', y='rate', ax=ax, label='Rate calculated from JPL HORIZONS ephemerides')
        newEphem.plot(x='datetime_str', y='startRate', ax=ax, label='Initial tracking rate')
        newEphem.plot(x='datetime_str', y='meanRate', ax=ax, label='Mean tracking rate')
        ax.set_ylabel("Tracking Rate [arcseconds per minute]")
        ax.xaxis.set_tick_params(labelbottom=False, which='both', direction='in', labelsize=14)

        frame2=fig.add_axes((.1,.1,.8,.2))
        ax = plt.gca()
        ax.set_xlabel("Universal Time [UT]")
        newEphem.plot(x='datetime_str', y='zero', ax=ax, label='Root Mean Square Residual = 0.00000')
        newEphem.plot(x='datetime_str', y='startRateRes', ax=ax, label='Root Mean Square Residual = %.5f' % rmseStart)
        newEphem.plot(x='datetime_str', y='meanRateRes', ax=ax, label='Root Mean Square Residual = %.5f' % rmseMean)
        ax.set_xlabel("Universal Time [UT]")
        ax.set_ylabel("Residuals [arcsec/min]")
        plt.savefig("%sHrProgram/trackingRates_%s.pdf" % (programLength, i), bbox_inches='tight')
        plt.close()
        
        rmseFile = open("%sHrProgram/temp_%s.csv" % (programLength, i), "w")
        rmseFile.write("updateTime,rmse\n")
        rmseArray = [programLength, rmseStart, rmseMean]
        for updateTime in (1,10,20,30,40,50,60):
        
            rate = []
            for j in range(0, len(newEphem.index)-updateTime, updateTime):
                rate.append((newEphem['RA'].iloc[j+updateTime] - newEphem['RA'].iloc[j])/updateTime)
            endIndex = len(newEphem.index)-1
            rate.append((newEphem['RA'].iloc[endIndex] - newEphem['RA'].iloc[endIndex-updateTime])/updateTime)
            
            
            updatingRate = []
            for k in range(0, len(newEphem.index)-1):
                updatingRate.append(rate[k//updateTime])
            updatingRate.append(rate[(len(newEphem.index)-1)//updateTime])

            newEphem['updatingRate'] = updatingRate
            
            newEphem['updatingRate'] = newEphem['updatingRate']*3600
            
            newEphem['Res'] = newEphem['rate'] - newEphem['updatingRate']
            rmse = np.sqrt(np.square(newEphem['Res']).sum())
            fig = plt.figure(figsize=(10,8))
            frame1=fig.add_axes((.1,.3,.8,.6))
            ax=plt.gca()

            newEphem.plot(x='datetime_str', y='rate', ax=ax, label='Tracking Rate calculated from JPL HORIZONS ephemerides')
            newEphem.plot(x='datetime_str', y='updatingRate', ax=ax, label='Tracking rate updated every %s mins' % updateTime)
            ax.set_ylabel("Tracking Rate [arcseconds per minute]")
            ax.xaxis.set_tick_params(labelbottom=False, which='both', direction='in', labelsize=14)

            frame2=fig.add_axes((.1,.1,.8,.2))
            ax = plt.gca()
            ax.set_xlabel("Universal Time [UT]")
            newEphem.plot(x='datetime_str', y='zero', ax=ax, label='Root Mean Square Residual = 0.00000')
            newEphem.plot(x='datetime_str', y='Res', ax=ax, label='Root Mean Square Residual = %.5f' % rmse)
            ax.set_xlabel("Universal Time [UT]")
            ax.set_ylabel("Residuals [arcsec/min]")
            plt.savefig("%sHrProgram/trackingRatesUpdating_%s_%s.pdf" % (programLength, i, updateTime), bbox_inches='tight')
            plt.close()
            rmseArray.append(rmse)

            rmseFile.write("%s,%s\n" % (updateTime,rmse))
            
        rmseFile.close()
        
        df = pd.read_csv(rmseFile.name)
        fig, ax = plt.subplots(figsize=(10,8))
        ax = plt.gca()
        df.plot(x='updateTime', y='rmse', ax=ax, color='k', marker='o', label='RMS Residual for varying update time')
        ax.axhline(y=rmseStart, color='k', linestyle='-.', label='RMS Residual [starting tracking rate]')
        ax.axhline(y=rmseMean, color='k', linestyle='--', label='RMS Residual [mean tracking rate]')
        ax.set_xlabel("Update Time [minutes]")
        ax.set_ylabel("RMS Residual [arcsec/min]")
        plt.legend()
        plt.savefig("%sHrProgram/rmsePlot_%s.pdf" % (programLength, i), bbox_inches='tight')
        plt.close()
        return rmseArray

In [3]:
f = open("targets.txt","r")        

array = []

for i in (1,2,3,4,5,6,7,8):
    start = (datetime.strptime("2019-11-17 00:00", "%Y-%m-%d %H:%M")-timedelta(hours=i/2)).strftime("%Y-%m-%d %H:%M")
    end = (datetime.strptime("2019-11-17 00:00", "%Y-%m-%d %H:%M")+timedelta(hours=i/2)).strftime("%Y-%m-%d %H:%M")
    rmseArray = generatePlotsRates('481394', start, end, i)
    array.append(rmseArray)

df = pd.DataFrame(array, columns=["ProgramLength","Start","Mean","1minute","10minute","20minute","30minute","40minute","50minute","60minute"])
#num_cores = multiprocessing.cpu_count()
#Parallel(n_jobs=num_cores)(delayed(generatePlotsRates)(obj) for obj in f)

fig, ax = plt.subplots(figsize=(10,8))
ax = plt.gca()
df.plot(x='ProgramLength', y='Start', ax=ax, marker='o', label='Starting tracking rate')
df.plot(x='ProgramLength', y='Mean', ax=ax, marker='o', label='Mean tracking rate')
df.plot(x='ProgramLength', y='1minute', ax=ax, marker='o', label='1 minute update time')
df.plot(x='ProgramLength', y='10minute', ax=ax, marker='o', label='10 minute update time')
df.plot(x='ProgramLength', y='20minute', ax=ax, marker='o', label='20 minute update time')
df.plot(x='ProgramLength', y='30minute', ax=ax, marker='o', label='30 minute update time')
df.plot(x='ProgramLength', y='40minute', ax=ax, marker='o', label='40 minute update time')
df.plot(x='ProgramLength', y='50minute', ax=ax, marker='o', label='50 minute update time')
df.plot(x='ProgramLength', y='60minute', ax=ax, marker='o', label='60 minute update time')
ax.set_xlabel("Program Length [hours]")
ax.set_ylabel("RMS Residual [arcsec/min]")
plt.legend()
plt.savefig("rmsePlot_programTime.pdf", bbox_inches='tight')
plt.close()

fig, ax = plt.subplots(figsize=(10,8))
ax = plt.gca()
df.plot(x='ProgramLength', y='Mean', ax=ax, marker='o', label='Mean tracking rate')
df.plot(x='ProgramLength', y='1minute', ax=ax, marker='o', label='1 minute update time')
df.plot(x='ProgramLength', y='10minute', ax=ax, marker='o', label='10 minute update time')
df.plot(x='ProgramLength', y='20minute', ax=ax, marker='o', label='20 minute update time')
df.plot(x='ProgramLength', y='30minute', ax=ax, marker='o', label='30 minute update time')
df.plot(x='ProgramLength', y='40minute', ax=ax, marker='o', label='40 minute update time')
df.plot(x='ProgramLength', y='50minute', ax=ax, marker='o', label='50 minute update time')
df.plot(x='ProgramLength', y='60minute', ax=ax, marker='o', label='60 minute update time')
ax.set_xlabel("Program Length [hours]")
ax.set_ylabel("RMS Residual [arcsec/min]")
plt.legend()
plt.savefig("rmsePlot_programTime_withoutStart.pdf", bbox_inches='tight')
plt.close()

f.close()